In [2]:

import pandas as pd
import numpy as np
import networkx as nx

Import edge table

In [ ]:
final_with_coordinates = pd.read_excel("edge.xlsx")


create new colomn called mode to indicate the transportation method

In [ ]:
final_with_coordinates["mode"] = "BUS"

In [ ]:
print(final_with_coordinates.head())

   ROUTE_ID  ROUTE_SEQ  ON_SEQ  OFF_SEQ  PRICE LAST_UPDATE_DATE_x  ON_STOP_ID  \
0      1001          1       1        2    6.7         2025-01-02        4001   
1      1001          1       1        3    6.7         2025-01-02        4001   
2      1001          1       1        4    6.7         2025-01-02        4001   
3      1001          1       1        5    6.7         2025-01-02        4001   
4      1001          1       1        6    6.7         2025-01-02        4001   

   OFF_STOP_ID  STOP_TYPE_x  X_ON_STOPID  Y_ON_STOPID LAST_UPDATE_DATE_y  \
0         4002            1       837872       822918         2023-03-04   
1         4003            1       837872       822918         2023-03-04   
2         4004            1       837872       822918         2023-03-04   
3         4005            1       837872       822918         2023-03-04   
4         4006            1       837872       822918         2023-03-04   

   STOP_CODE_x  STOP_TYPE_y  X_OFF_STOPID  Y_OFF_STOPID 

create new edge (by walking) if the distance between 2 stops are less than 250(around 5 mminutes walk)

In [ ]:
unique_stops = final_with_coordinates[["ON_STOP_ID", "X_ON_STOPID", "Y_ON_STOPID"]].drop_duplicates()

# Calculate pairwise distances
stop_pairs = []
for i, stop1 in unique_stops.iterrows():
    for j, stop2 in unique_stops.iterrows():
        if i < j:  # Avoid duplicate calculations
            distance = np.sqrt(
                (stop1["X_ON_STOPID"] - stop2["X_ON_STOPID"]) ** 2 +
                (stop1["Y_ON_STOPID"] - stop2["Y_ON_STOPID"]) ** 2
            )
            if distance < 250:
                # Add both combinations of the pair
                stop_pairs.append(
                    {
                        "ON_STOP_ID": stop1["ON_STOP_ID"],
                        "OFF_STOP_ID": stop2["ON_STOP_ID"],
                        "DISTANCE": distance,
                    }
                )
                stop_pairs.append(
                    {
                        "ON_STOP_ID": stop2["ON_STOP_ID"],
                        "OFF_STOP_ID": stop1["ON_STOP_ID"],
                        "DISTANCE": distance,
                    }
                )

# Create a DataFrame for the new "walk" rows
new_rows = []
for pair in stop_pairs:
    new_rows.append({
        "ROUTE_ID": None,
        "ROUTE_SEQ": None,
        "ON_SEQ": None,
        "OFF_SEQ": None,
        "PRICE": 2,
        "LAST_UPDATE_DATE_x": None,
        "ON_STOP_ID": pair["ON_STOP_ID"],
        "OFF_STOP_ID": pair["OFF_STOP_ID"],
        "STOP_TYPE_x": None,
        "X_ON_STOPID": None,
        "Y_ON_STOPID": None,
        "LAST_UPDATE_DATE_y": None,
        "STOP_CODE_x": None,
        "STOP_TYPE_y": None,
        "X_OFF_STOPID": None,
        "Y_OFF_STOPID": None,
        "LAST_UPDATE_DATE": None,
        "STOP_CODE_y": None,
        "DISTANCE": pair["DISTANCE"],
        "mode": "walk",  # New column
    })

# Append the new rows to the original DataFrame
new_rows_df = pd.DataFrame(new_rows)
updated_df = pd.concat([final_with_coordinates, new_rows_df], ignore_index=True)

# Display the updated DataFrame
print(updated_df)

<ipython-input-8-22a5ee1dbfcf>:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([final_with_coordinates, new_rows_df], ignore_index=True)


       ROUTE_ID ROUTE_SEQ ON_SEQ OFF_SEQ  PRICE LAST_UPDATE_DATE_x  \
0          1001         1      1       2    6.7         2025-01-02   
1          1001         1      1       3    6.7         2025-01-02   
2          1001         1      1       4    6.7         2025-01-02   
3          1001         1      1       5    6.7         2025-01-02   
4          1001         1      1       6    6.7         2025-01-02   
...         ...       ...    ...     ...    ...                ...   
778900     None      None   None    None    2.0                NaT   
778901     None      None   None    None    2.0                NaT   
778902     None      None   None    None    2.0                NaT   
778903     None      None   None    None    2.0                NaT   
778904     None      None   None    None    2.0                NaT   

        ON_STOP_ID  OFF_STOP_ID STOP_TYPE_x X_ON_STOPID Y_ON_STOPID  \
0             4001         4002           1      837872      822918   
1             400

import tbale realted to the destination

In [37]:
filtered_df = pd.read_excel('weare.xlsx')

destination_stops = set(filtered_df["STOP_ID"])

# Step 2: Extract Unique Stops
unique_stops = set(updated_df["ON_STOP_ID"]).union(set(updated_df["OFF_STOP_ID"]))



Start building the map to find optimal path based on price

In [ ]:
import networkx as nx

# Use MultiDiGraph to accommodate multiple edges between nodes
G = nx.MultiDiGraph()

# Add edges to the graph, including the "mode" attribute
for _, row in updated_df.iterrows():
    G.add_edge(
        row["ON_STOP_ID"],
        row["OFF_STOP_ID"],
        distance=row["DISTANCE"],
        price=row["PRICE"],  # Price already includes walking cost if mode == "WALK"
        mode=row["mode"],    # Add mode attribute (either "BUS" or "WALK")
    )

result_data = []

for stop in unique_stops:
    if stop in destination_stops:
        continue  # Skip destination stops as starting points

    # Use Dijkstra's algorithm to find the shortest path to any destination stop
    try:
        # Compute shortest paths to all destinations
        shortest_path = None
        shortest_price = float("inf")
        shortest_distance = 0
        total_walking_cost = 0

        for dest in destination_stops:
            if nx.has_path(G, stop, dest):
                # Compute the shortest path based on the "price" weight
                path_price = nx.shortest_path_length(G, source=stop, target=dest, weight="price")

                # Skip paths where the price exceeds $60
                if path_price > 60:
                    continue

                if path_price < shortest_price:
                    shortest_price = path_price
                    shortest_path = nx.shortest_path(G, source=stop, target=dest, weight="price")

                    # Calculate total distance for this path
                    shortest_distance = sum(
                        G[u][v][key]["distance"]
                        for u, v in zip(shortest_path[:-1], shortest_path[1:])
                        for key in G[u][v]  # Iterate over all edges between u and v
                    )

                    # Calculate total walking cost for this path
                    total_walking_cost = sum(
                        G[u][v][key]["price"]
                        for u, v in zip(shortest_path[:-1], shortest_path[1:])
                        for key in G[u][v]
                        if G[u][v][key]["mode"] == "walk"
                    )

        if shortest_path:
            # Add to results
            result_data.append({
                "ON_STOP_ID": stop,
                "OFF_STOP_ID": shortest_path[-1],
                "TOTAL_PRICE": shortest_price,
                "TOTAL_DISTANCE": shortest_distance,
                "TOTAL_WALKING_COST": total_walking_cost,
                "PATH": shortest_path,
            })

    except nx.NetworkXNoPath:
        # No valid path to any destination stop
        continue
result_df = pd.DataFrame(result_data)

Convert colomn path to a list

In [9]:
import ast
result_df['PATH'] = result_df['PATH'].apply(ast.literal_eval)

In [24]:

nan_rows = updated_df[updated_df[['ROUTE_ID', 'ON_SEQ', 'OFF_SEQ']].isna().any(axis=1)]
print(nan_rows)
print(updated_df.isna().sum())

        ROUTE_ID  ROUTE_SEQ  ON_SEQ  OFF_SEQ  PRICE LAST_UPDATE_DATE_x  \
755971       NaN        NaN     NaN      NaN    2.0                NaN   
755972       NaN        NaN     NaN      NaN    2.0                NaN   
755973       NaN        NaN     NaN      NaN    2.0                NaN   
755974       NaN        NaN     NaN      NaN    2.0                NaN   
755975       NaN        NaN     NaN      NaN    2.0                NaN   
...          ...        ...     ...      ...    ...                ...   
778900       NaN        NaN     NaN      NaN    2.0                NaN   
778901       NaN        NaN     NaN      NaN    2.0                NaN   
778902       NaN        NaN     NaN      NaN    2.0                NaN   
778903       NaN        NaN     NaN      NaN    2.0                NaN   
778904       NaN        NaN     NaN      NaN    2.0                NaN   

        ON_STOP_ID  OFF_STOP_ID  STOP_TYPE_x  X_ON_STOPID  Y_ON_STOPID  \
755971        4001          793      

Find the true distance(some error occur on the map building function):

In [56]:

total_distances = []
total_prices = [] # To store total distances for each path
finsihed = 0

updated_df_nonan = updated_df.dropna(subset=['ROUTE_ID', 'ON_SEQ', 'OFF_SEQ'])

for _, row in result_df.iterrows():
    path = row['PATH']  # Get the path list
    total_distance = 0
    total_price = 0  # Variable to store the sum of distances for the current path

    # Loop through consecutive pairs in the path
    for i in range(len(path) - 1):
        on_stop = path[i]
        off_stop = path[i + 1]
        price = 0
        distance = 0

        distance_rows = updated_df[(updated_df['ON_STOP_ID'].astype(int) == int(on_stop)) & (updated_df['OFF_STOP_ID'].astype(int) == int(off_stop))]

        if (distance_rows['mode'] == 'walk').any():
    # Directly add price and distance
            min_price_row = distance_rows.loc[distance_rows['PRICE'].idxmin()]
            distance = min_price_row["DISTANCE"]
            price = min_price_row['PRICE']  # The target OFF_SEQ to match in the loop
            total_price += price
            total_distance += distance
            print("this is a walk")
            continue

        if not distance_rows.empty:
            # Find the row with the lowest price
            min_price_row = distance_rows.loc[distance_rows['PRICE'].idxmin()]

            # Extract ROUTE_ID, ON_SEQ, and OFF_SEQ from the lowest price row
            route_id = int(min_price_row['ROUTE_ID'])
            on_seq = int(min_price_row['ON_SEQ'])
            off_seq_target = int(min_price_row['OFF_SEQ'])
            print(off_seq_target)
            price = min_price_row['PRICE']  # The target OFF_SEQ to match in the loop
            total_price += price
            # Initialize variables for the loop
            current_on_seq = on_seq
            while True:
                # Find the row with the same ROUTE_ID and current ON_SEQ
                next_row = updated_df_nonan[(updated_df_nonan['ROUTE_ID'].astype(int) == int(route_id)) &
                                      (updated_df_nonan['ON_SEQ'].astype(int) == int(current_on_seq))&
                                     (updated_df_nonan['OFF_SEQ'].astype(int) > int(current_on_seq)) ]

                if next_row.empty:
                    break  # No matching row found, exit the loop

                # Extract OFF_SEQ and DISTANCE from the current row
                current_off_seq = int(next_row['OFF_SEQ'].min())
                distance = float(next_row.iloc[0]['DISTANCE'])
                #take one sample to check whether the code is correct
                if(on_stop == 10000008 and off_stop == 929):
                  print(route_id)
                  print(current_on_seq)
                  print(current_off_seq)
                  print(distance)
                total_distance += distance
                # Check if we've reached the target OFF_SEQ
                if current_off_seq >= off_seq_target:
                    break  # Exit the loop when the target OFF_SEQ is reached

                # Increment ON_SEQ for the next iteration
                current_on_seq = current_off_seq

    # Append the total distance for this path to the list
    total_distances.append(total_distance)
    total_prices.append(total_price)
    finsihed += 1
    print(str(finsihed)+ "/4380")

# Add the total distances as a new column in result_df
result_df['total_distance'] = total_distances
result_df['total new price'] = total_prices

串流輸出內容已截斷至最後 5000 行。
this is a walk
this is a walk
16
2719/4380
this is a walk
25
21
2720/4380
18
2721/4380
13
2722/4380
13
16
2723/4380
this is a walk
13
2724/4380
this is a walk
18
2725/4380
this is a walk
18
2726/4380
this is a walk
13
25
2727/4380
21
2728/4380
this is a walk
10
2729/4380
this is a walk
this is a walk
16
2730/4380
18
43
2731/4380
this is a walk
13
2732/4380
12
21
2733/4380
this is a walk
18
2734/4380
this is a walk
18
2735/4380
this is a walk
28
17
2736/4380
this is a walk
18
2737/4380
25
21
2738/4380
this is a walk
18
2739/4380
this is a walk
13
2740/4380
13
2741/4380
this is a walk
18
2742/4380
15
19
2743/4380
this is a walk
18
2744/4380
this is a walk
16
2745/4380
this is a walk
18
2746/4380
this is a walk
18
2747/4380
7
18
2748/4380
this is a walk
18
2749/4380
this is a walk
this is a walk
21
2750/4380
this is a walk
24
19
2751/4380
this is a walk
16
2752/4380
this is a walk
13
2753/4380
25
this is a walk
2754/4380
this is a walk
25
2755/4380
this is a walk
15
1

Check whether the path are accuarate

In [ ]:
result_df['ON_STOP_MATCH'] = result_df.apply(lambda row: row['ON_STOP_ID'] == row['PATH'][0], axis=1)
result_df['OFF_STOP_MATCH'] = result_df.apply(lambda row: row['OFF_STOP_ID'] == row['PATH'][-1], axis=1)

# Combine both conditions into a single column if needed
result_df['BOTH_MATCH'] = result_df['ON_STOP_MATCH'] & result_df['OFF_STOP_MATCH']

print(sum(result_df['BOTH_MATCH']))
result_df = result_df.drop(columns=['TOTAL_DISTANCE', 'TOTAL_WALKING_COST'])

export the file

In [57]:
from google.colab import files


# Display the result
print(result_df)
# Step 1: Save the DataFrame to a CSV file
result_df.to_csv('result.csv', index=False)  # Save without the index column

# Step 2: Download the file
files.download('result.csv')



      ON_STOP_ID  OFF_STOP_ID  TOTAL_PRICE  TOTAL_DISTANCE  \
0       10000007          130         28.2   224191.592000   
1          12589          130         28.2   224089.290000   
2          12868          130         28.2   224088.123900   
3       10000008          130         26.2   224062.123900   
4          11000          130         35.3   203638.934500   
...          ...          ...          ...             ...   
4381         425          233          2.0      242.738542   
4382         616          518          2.0      235.983050   
4383         319          130          2.0      227.554389   
4384       13071          130          2.0      185.270073   
4385         467          130          2.0      118.139748   

      TOTAL_WALKING_COST                             PATH  new cost  \
0                      2   [10000007, 10000008, 929, 130]      26.2   
1                      2      [12589, 10000008, 929, 130]      26.2   
2                      2      [12868, 1000

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Pick some rows to check for validity

In [76]:
[ 7080, 8050]
distance_rows = updated_df[(updated_df['ON_STOP_ID'].astype(int) == 7080) & (updated_df['OFF_STOP_ID'].astype(int) == 8050)]
distance_rows = distance_rows[distance_rows["ROUTE_ID"] == 1270.0]
route_id = distance_rows["ROUTE_ID"]
print(distance_rows)
target_off_seq = distance_rows["OFF_SEQ"]
current_on_seq = distance_rows["ON_SEQ"]
print(current_off_seq)
print(current_on_seq)
total_distance = 0
while True:
                # Find the row with the same ROUTE_ID and current ON_SEQ
    next_row = updated_df_nonan[(updated_df_nonan['ROUTE_ID'].astype(int) == int(route_id)) &
                                      (updated_df_nonan['ON_SEQ'].astype(int) == int(current_on_seq))&
                                     (updated_df_nonan['OFF_SEQ'].astype(int) > int(current_on_seq)) ]


    if next_row.empty:
        break  # No matching row found, exit the loop

                # Extract OFF_SEQ and DISTANCE from the current row
    current_off_seq = int(next_row['OFF_SEQ'].min())
    distance = float(next_row.iloc[0]['DISTANCE'])
                #take one sample to check whether the code is correct

    print(route_id)
    print(current_on_seq)
    print(current_off_seq)
    print(distance)
    total_distance += distance
                # Check if we've reached the target OFF_SEQ
    if current_off_seq >= int(target_off_seq):
        print("reached")
        break  # Exit the loop when the target OFF_SEQ is reached

                # Increment ON_SEQ for the next iteration
    current_on_seq = current_off_seq
print(total_distance)



        ROUTE_ID  ROUTE_SEQ  ON_SEQ  OFF_SEQ  PRICE LAST_UPDATE_DATE_x  \
101086    1270.0        1.0     4.0     16.0   13.8         2024-12-02   

        ON_STOP_ID  OFF_STOP_ID  STOP_TYPE_x  X_ON_STOPID  Y_ON_STOPID  \
101086        7080         8050          1.0     839382.0     820687.0   

       LAST_UPDATE_DATE_y  STOP_CODE_x  STOP_TYPE_y  X_OFF_STOPID  \
101086         2023-03-04          NaN          1.0      835064.0   

        Y_OFF_STOPID LAST_UPDATE_DATE  STOP_CODE_y    DISTANCE mode  
101086      815629.0       2023-03-04          NaN  6650.45021  BUS  
37
101086    4.0
Name: ON_SEQ, dtype: float64
101086    1270.0
Name: ROUTE_ID, dtype: float64
101086    4.0
Name: ON_SEQ, dtype: float64
5
2268.211850775849
101086    1270.0
Name: ROUTE_ID, dtype: float64
5
6
232.2455596992115
101086    1270.0
Name: ROUTE_ID, dtype: float64
6
7
1121.357213380286
101086    1270.0
Name: ROUTE_ID, dtype: float64
7
8
456.347455345157
101086    1270.0
Name: ROUTE_ID, dtype: float64
8
9
408.5

<ipython-input-76-6d67af3520b6>:13: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  next_row = updated_df_nonan[(updated_df_nonan['ROUTE_ID'].astype(int) == int(route_id)) &
<ipython-input-76-6d67af3520b6>:14: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  (updated_df_nonan['ON_SEQ'].astype(int) == int(current_on_seq))&
<ipython-input-76-6d67af3520b6>:15: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  (updated_df_nonan['OFF_SEQ'].astype(int) > int(current_on_seq)) ]
<ipython-input-76-6d67af3520b6>:32: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if current_off_seq >= int(target_off_seq):
<ipython-input-76-6d67af3520